# PRÁCTICA FINAL

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [126]:
df = pd.read_csv('./Unemployment in America Per US State.csv')
df.head()

,FIPS Code,State/Area,Year,Month,Total Civilian Non-Institutional Population in State/Area,Total Civilian Labor Force in State/Area,Percent (%) of State/Area's Population,Total Employment in State/Area,Percent (%) of Labor Force Employed in State/Area,Total Unemployment in State/Area,Percent (%) of Labor Force Unemployed in State/Area
0,1,Alabama,1976,1,"2,605,000","1,484,555",57.0,"1,386,023",53.2,"98,532",6.6
1,2,Alaska,1976,1,"232,000","160,183",69.0,"148,820",64.1,"11,363",7.1
2,4,Arizona,1976,1,"1,621,000","964,120",59.5,"865,871",53.4,"98,249",10.2
3,5,Arkansas,1976,1,"1,536,000","889,044",57.9,"824,395",53.7,"64,649",7.3
4,6,California,1976,1,"15,621,000","9,774,280",62.6,"8,875,685",56.8,"898,595",9.2


Lo primero que haremos será comprender los datos de los que disponemos. Contamos con 11 variables:


- FIPS Code of State/Area: identificador del Estado
- Year: año
- Month: mes
- Total Civilian Non-Institutional Population in State/Area: Número total de civiles no institucionalizados que residen en el estado/zona.
- Total Civilian Labor Force in State/Area: Número total de civiles aptos para el empleo que residen en el estado/zona.
- Percent (%) of State/Area's Population: Porcentaje de civiles aptos para el empleo sobre el total de la población civil no institucionalizada.
- Total Employment in State/Area: Número total de civiles empleados actualmente en el estado/zona: Porcentaje de civiles actualmente empleados sobre el total de la población civil no institucionalizada.
- Percent (%) of Labor Force Employed in State/Area: Porcentaje de civiles actualmente empleados sobre el total de la población civil no institucionalizada.
- Total Unemployment in State/Area: Número total de civiles actualmente desempleados en el estado/zona.
- Percent (%) of Labor Force Unemployed in State/Area: Porcentaje de civiles actualmente desempleados sobre el total de la población civil no institucionalizada.

## PREPROCESO

Primero comprobamos los tipos de datos que tenemos y la presencia de nulos

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29892 entries, 0 to 29891
Data columns (total 11 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   FIPS Code                                                  29892 non-null  int64  
 1   State/Area                                                 29892 non-null  object 
 2   Year                                                       29892 non-null  int64  
 3   Month                                                      29892 non-null  int64  
 4   Total Civilian Non-Institutional Population in State/Area  29892 non-null  object 
 5   Total Civilian Labor Force in State/Area                   29892 non-null  object 
 6   Percent (%) of State/Area's Population                     29892 non-null  float64
 7   Total Employment in State/Area                             29892 non-null  object 
 8   Percen

Observamos que todas las estadísticas civiles se almacenan con un tipo de dato objeto. Necesitamos convertirlas a un tipo de dato numérico para poder trabajar con ellas:

In [128]:
def objToInt(col):
    if df.dtypes[i]!='int64' and df.dtypes[i]!='float64':
        df[col]=df[col].str.replace(',','').astype(int)

In [129]:
columns = df.columns
for i in range(2,len(columns)):
    objToInt(columns[i])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29892 entries, 0 to 29891
Data columns (total 11 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   FIPS Code                                                  29892 non-null  int64  
 1   State/Area                                                 29892 non-null  object 
 2   Year                                                       29892 non-null  int64  
 3   Month                                                      29892 non-null  int64  
 4   Total Civilian Non-Institutional Population in State/Area  29892 non-null  int32  
 5   Total Civilian Labor Force in State/Area                   29892 non-null  int32  
 6   Percent (%) of State/Area's Population                     29892 non-null  float64
 7   Total Employment in State/Area                             29892 non-null  int32  
 8   Percen

Para poder trabajar de mejor forma con las series temporales, creamos una columna con la fecha (en formato año/mes/día) combinando el mes y el año, para tener una marca de tiempo en una única columna:

In [130]:
df['Date']=pd.to_datetime(df['Year']*100 + df['Month'] ,format='%Y%m')
# df.drop(columns=['Year','Month'],inplace=True)
df.head()

,FIPS Code,State/Area,Year,Month,Total Civilian Non-Institutional Population in State/Area,Total Civilian Labor Force in State/Area,Percent (%) of State/Area's Population,Total Employment in State/Area,Percent (%) of Labor Force Employed in State/Area,Total Unemployment in State/Area,Percent (%) of Labor Force Unemployed in State/Area,Date
0,1,Alabama,1976,1,2605000,1484555,57.0,1386023,53.2,98532,6.6,1976-01-01
1,2,Alaska,1976,1,232000,160183,69.0,148820,64.1,11363,7.1,1976-01-01
2,4,Arizona,1976,1,1621000,964120,59.5,865871,53.4,98249,10.2,1976-01-01
3,5,Arkansas,1976,1,1536000,889044,57.9,824395,53.7,64649,7.3,1976-01-01
4,6,California,1976,1,15621000,9774280,62.6,8875685,56.8,898595,9.2,1976-01-01


In [131]:
def extraer_serie_temporal(df,estado):
    if estado != 'USA':
        serie_temporal = df[df['State/Area']==estado]
        desempleo = serie_temporal['Total Unemployment in State/Area'].values*100
        poblacion = serie_temporal['Total Civilian Labor Force in State/Area'].values
        serie_temporal[str('% Desempleo en '+ estado)] = desempleo/poblacion

        sns.set(style='whitegrid')
        sns.lineplot(x = "Date",
                y = str('% Desempleo en '+ estado),
                data = serie_temporal)

        return serie_temporal
    else:
        desempleo=pd.DataFrame(df.groupby('Date')[["Total Unemployment in State/Area"]].sum())
        poblacion=pd.DataFrame(df.groupby('Date')[["Total Civilian Labor Force in State/Area"]].sum())

        desempleo.reset_index(inplace=True)
        poblacion.reset_index(inplace=True)

        desempleo['Total Unemployment in State/Area']*=100

        serie_temporal=pd.DataFrame()
        serie_temporal['Date'] = df['Date'].unique()
        serie_temporal['% Unemployment in US'] = desempleo['Total Unemployment in State/Area']/poblacion["Total Civilian Labor Force in State/Area"]
        sns.set(style='whitegrid')

        sns.lineplot(x = "Date",
             y = "% Unemployment in US",
             data = serie_temporal)
        return serie_temporal